# 高速な Julia


多くの場合, ベンチマークを使用して言語を比較する. これらのベンチマークは, 最初に正確に何がベンチマークされているのか, そして次にその違いを説明しているのかについて, 長い議論につながる可能性がある. これらシンプルな質問は, あなたが想像しているよりも複雑な問になることがある.

このノートブックの目的は, シンプルなベンチマークを見ることである. ノートブックをよみ, 作者のMacbook Pro, 4-core Intel Core I7 で何が起こったかを確認したり, ノートブックを実行することができる. 

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# 概要

- `sum` 関数を定義する
- `sum` 関数を下記で実装しベンチマークを取る...
    - C
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
- ベンチマーク要約

# `sum`: 理解のための十分な関数

下記を計算する **sum** 関数 `sum(a)` を考えてみよう.

$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$

$n$ は `a` の長さである.

In [3]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.412376 
 0.0266054
 0.860611 
 0.348417 
 0.920091 
 0.410966 
 0.480448 
 0.0100622
 0.38225  
 0.669933 
 0.760516 
 0.670021 
 0.527722 
 ⋮        
 0.178003 
 0.817984 
 0.506671 
 0.936143 
 0.242308 
 0.0388534
 0.294296 
 0.826521 
 0.0416863
 0.684282 
 0.957743 
 0.267376 

In [4]:
sum(a)   

4.999167623462351e6

各要素の平均は 0.5 であるため, 期待される結果は, $0.5 \times 10^7$ である.

# いくつかの言語にたいして, いくつかの方法でベンチマークを取る

Julia には `BenchmarkTools.jl` パッケージがあり、簡単で正確なベンチマークが可能である:

In [6]:
#Pkg.add("BenchmarkTools")

INFO: Cloning cache of BenchmarkTools from https://github.com/JuliaCI/BenchmarkTools.jl.git
INFO: Installing BenchmarkTools v0.2.5
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of BenchmarkTools
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [7]:
using BenchmarkTools  

INFO: Precompiling module BenchmarkTools.


#  1. Ｃ言語

C言語は,しばしばゴールドスタンダードとみなされる: 人間にとっては難しく, マシンにとっては適している. C言語の2倍の範囲内で処理ができれば, 十分である. 
それにもかかわらず, Ｃ言語の範囲内であっても、ネイティブC言語プログラマが優位を得られるかどうかはさまざまな種類の最適化がもとめられる.

現著者は, C言語を書かないので, 下記のセルのコードはわからない, しかし, Julia セッションにおいて C 言語コードをかけて, コンパイルできて, 実行できるのはうれしいことだと思う.

`"""` トリプルダブルクオーテーションで囲むことを注意してほしい.

In [8]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file

# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [9]:
c_sum(a)

4.999167623462151e6

In [10]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

In [11]:
≈  # alias for the `isapprox` function

isapprox (generic function with 3 methods)

In [12]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=sqrt(eps), atol::Real=0, nans::Bool=false, norm::Function)
```

Inexact equality comparison: `true` if `norm(x-y) <= atol + rtol*max(norm(x), norm(y))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, `rtol` defaults to `sqrt(eps(typeof(real(x-y))))`. This corresponds to requiring equality of about half of the significand digits. For other types, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to `vecnorm` but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true
```


We can now benchmark the C code directly from Julia:

In [13]:
c_bench = @benchmark c_sum($a) 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.358 ms (0.00% GC)
  median time:      8.996 ms (0.00% GC)
  mean time:        9.226 ms (0.00% GC)
  maximum time:     12.390 ms (0.00% GC)
  --------------
  samples:          541
  evals/sample:     1

In [14]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 8.358275 msec


In [15]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 1 entry:
  "C" => 8.35828

# 2. Pythonのビルトイン関数 `sum` 

`PyCall` パッケージは, Python への Julia インタフェースを提供する:

In [ ]:
#Pkg.add("PyCall")

In [16]:
using PyCall

LoadError: [91mArgumentError: Module PyCall not found in current path.
Run `Pkg.add("PyCall")` to install the PyCall package.[39m

In [ ]:
# Call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

In [ ]:
pysum(a)

In [ ]:
pysum(a) ≈ sum(a)

In [ ]:
py_list_bench = @benchmark $pysum($apy_list)

In [ ]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

# 3. Python: `numpy` 

## "SIMD" ハードウェアが使える場合は活用する.

`numpy` は, python からよばれる最適化された `C` ライブラリである.
次のように, Julia にインストールできる:


In [ ]:
using Conda 
#Conda.add("numpy")

In [ ]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

In [ ]:
numpy_sum(apy_list) # python thing

In [ ]:
numpy_sum(apy_list) ≈ sum(a)

In [ ]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

# 4. Python, 自作関数 

In [ ]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s += x
    return s
"""

sum_py = py"py_sum"

In [ ]:
py_hand = @benchmark $sum_py($apy_list)

In [ ]:
sum_py(apy_list)

In [ ]:
sum_py(apy_list) ≈ sum(a)

In [ ]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

# 5. Julia のビルトイン関数

## C言語ではなく, Julia で書かれている

In [ ]:
@which sum(a)

In [ ]:
j_bench = @benchmark sum($a)

In [ ]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

# 6. Julia 自作関数

In [ ]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A
        s = s + a
    end
    s
end

In [ ]:
j_bench_hand = @benchmark mysum($a)

In [ ]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

# ベンチマーク要約

In [17]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 20, "."), lpad(round(value, 2), 10, "."))
end

C.........................8.36
